In [11]:
from urllib.request import urlopen 
from urllib.parse import urlencode, quote_plus
import requests

import xmltodict
import json
import requests
from io import StringIO

import geopandas as gpd
from shapely.geometry import Polygon
from shapely import wkt

import pandas as pd
import numpy as np

from datetime import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

import osmnx as ox
import networkx as nx
from haversine import haversine

from sklearn.preprocessing import MinMaxScaler
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.metrics.pairwise import haversine_distances
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, PULP_CBC_CMD

In [ ]:
# csv 파일 부르기
fire_station = pd.read_csv('data/firestation.csv') 
pop = pd.read_csv('data/hap_pop.csv',encoding='cp949') 
forest_area = gpd.read_file('data/hap_tree.zip') 

### 대상지 선정 (공공데이터포털 산불 API 활용)

In [ ]:
data=[]
for i in range(2014, 2024):
    key=''
    url=f'http://apis.data.go.kr/1400000/forestStusService/getfirestatsservice?serviceKey={key}&'
    queryParams =urlencode({quote_plus('pageNo') : '1',quote_plus('numOfRows') : '9999',
                            quote_plus('searchStDt') : f'{i}0101',quote_plus('searchEdDt') : f'{i}1231'})

    url2=url+queryParams
    response = urlopen(url2)
    results = response.read().decode("utf-8")
    results_to_json = xmltodict.parse(results)
    d = json.loads(json.dumps(results_to_json))
    data.extend(d['response']['body']['items']['item'])

df=pd.DataFrame(data)
si_do={'강원':'Gangwon-do','경기':'Gyeonggi-do', '경남':'Gyeongsangnam-do','경북':'Gyeongsangbuk-do',
       '광주':'Gwangju','대구':'Daegu','대전':'Daejeon','부산':'Busan',
       '서울':'Seoul','세종':'Sejongsi','울산':'Ulsan','인천':'Incheon',
       '전남':'Jeollanam-do','전북':'Jeollabuk-do','제주':'Jeju-do','충남':'Chungcheongnam-do',
       '충북':'Chungcheongbuk-do'}

df['eng_region']=df['locsi'].apply(lambda x: si_do[x])

In [ ]:
df_size = df.groupby('locsi').size()
df_size = df_size.reset_index()
df_size.columns = ['시도', '산불건수']

df_sorted = df_size.sort_values(by='산불건수', ascending=False)

plt.rc('font', family='NanumBarunGothic')
plt.figure(figsize=(12, 6))
sns.barplot(data=df_sorted, x='시도', y='산불건수', palette=[
    'red' if x == '경남' else 'gray' for x in df_sorted['시도']
])
plt.title('전국 시도별 산불 발생 건수 (최근 10년)', fontsize=14)
plt.ylabel('건수')
plt.xlabel('시도')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df_gn = df[df['locsi'] == '경남'].reset_index(drop=True)

df_gn.dropna(subset=['locgungu'], inplace=True)
df_gn['locgungu'] = df_gn['locgungu'].apply(lambda x : '창원' if isinstance(x, str) and '창원' in x else x)

In [ ]:
fire_freq_area = df_gn.groupby('locgungu').size().reset_index(name='fire_count')

df_gn['log_damagearea'] = np.log1p(df_gn['damagearea'])
fire_freq_area['total_area'] = fire_freq_area['locgungu'].map(df_gn.groupby('locgungu')['log_damagearea'].sum())

scale = MinMaxScaler()
fire_freq_area[['fire_count_scaled', 'total_area_scaled']] = scale.fit_transform(fire_freq_area[['fire_count', 'total_area']])
fire_freq_area['risk_score'] = fire_freq_area['fire_count_scaled']*0.5 + fire_freq_area['total_area_scaled']*0.5
fire_freq_area['risk_score_100'] = fire_freq_area['risk_score']*100

# risk score를 이용한 버블차트 >> 최종적으로 합천 선택
plt.rc('font', family='Malgun Gothic') 
plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=fire_freq_area.sort_values('risk_score'),
    x='locgungu',         
    y='total_area',         
    size='risk_score',      
    hue='risk_score',        
    sizes=(100,2000),
    palette='Reds',
    legend=None
)

plt.title('시군별 산불 빈도와 피해 면적에 따른 시각화', fontsize=16)
plt.xlabel('시군', fontsize=12)
plt.ylabel('피해 면적', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
df_hapcheon = df_gn[df_gn['locgungu'] == '합천'].reset_index(drop=True)
df_hapcheon = df_hapcheon[['startyear','damagearea', 'locmenu']]
#df_hapcheon.to_csv('df_hapcheon', index = False)

### 읍면동 중심좌표(경계)

In [ ]:
# 공공데이터포털 (통계청_SGIS 행정구역 통계 및 경계) 자료 이용
shapefile_path = ''
gdf = gpd.read_file(shapefile_path)

# 합천군 코드 : 38600으로 시작
hapcheon_gdf = gdf[gdf['ADM_CD'].str.startswith('38600')].copy()
hapcheon_gdf.drop('BASE_DATE', axis=1, inplace=True)
hapcheon_gdf = hapcheon_gdf.reset_index(drop= True)

hapcheon_gdf = hapcheon_gdf.to_crs(epsg=5186)
hapcheon_gdf['centroid'] = hapcheon_gdf.geometry.centroid
hapcheon_gdf.set_geometry('centroid', inplace=True)

# 위도/ 경도로 변환
hapcheon_centroid = hapcheon_gdf.to_crs(epsg=4326)
hapcheon_centroid['ADM_NM'] = hapcheon_centroid['ADM_NM'].str[:-1] 
hapcheon_centroid['lon'] = hapcheon_centroid.geometry.x
hapcheon_centroid['lat'] = hapcheon_centroid.geometry.y
# hapcheon_centroid.to_csv('hapcheon_centroid.csv', index= False)

### 위험도(빈도, 피해 면적) 계산

In [ ]:
df_hap = df_hapcheon.copy()
hap_center = hapcheon_centroid.copy()

In [ ]:
# 빈도
df_hap['freq'] = df_hap['startyear'].apply(lambda x : 0.6 if x >= 2021 else 0.4)
year_freq = df_hap.groupby('locmenu')['freq'].sum().reset_index()
year_freq.columns = ['ADM_NM', 'year_score']

scale = MinMaxScaler()
year_freq['year_score_scaled'] = scale.fit_transform(year_freq[['year_score']])

In [ ]:
# 피해면적
scale = MinMaxScaler()

df_hap['damagearea_scaled'] = np.log1p(df_hap['damagearea'])
df_hap['damagearea_scaled'] = scale.fit_transform(df_hap[['damagearea_scaled']])

damage_area = df_hap.groupby('locmenu')['damagearea'].sum().reset_index()
damage_area.columns = ['ADM_NM', 'damagearea']

scale = MinMaxScaler()
damage_area['damagearea_scaled'] = np.log1p(damage_area['damagearea'])
damage_area['damagearea_scaled'] = scale.fit_transform(damage_area[['damagearea_scaled']])

### 위험도(자연요소) 계산 (기상청API 활용)

In [ ]:
key = ''
wind_url = f'https://apihub.kma.go.kr/api/typ01/url/sts_wind.php?authKey={key}'

wind_data = []
startyear = 2014
endyear = 2023

for year in range(startyear, endyear + 1):
    for month in range(1, 13):
        tm1 = f'{year}{month:02d}'  # 예: 201401
        tm2 = f'{year}{month:02d}'  # 예: 201401

        # 파라미터 설정
        params = {
            'tm1': tm1,
            'tm2': tm2,
            'stn_id': 0,  # 전국 데이터
            'disp': 1,
            'help': 0
        }
        response = requests.get(wind_url, params=params)
        # StringIO를 이용해 응답 텍스트를 pandas로 읽기
        # 응답 데이터가 CSV 형태로 제공된다고 가정하고 StringIO로 읽기
        if response.text:
            df = pd.read_csv(StringIO(response.text), sep='\s+')  # CSV 형식으로 읽기
            # 데이터에 연도와 월을 추가
            df['Year'] = year
            df['Month'] = month
            wind_data.append(df)  # DataFrame 리스트에 추가
        else:
            print(f"Empty data for {year}-{month:02d}")

wind_df = pd.concat(wind_data, ignore_index=True)
# wind_df.to_csv('wind_df.csv',index=False)

In [ ]:
# 합천 기상관측소 바람 데이터 불러오기 (대표 관측소)
hap_wind = wind_df[(wind_df['STN_ID']==285) & (wind_df['lon'] == 35.56505)].reset_index(drop=True)

mean_wsmax = hap_wind[hap_wind.WS_MAX >= hap_wind.WS_MAX.quantile(0.9)]['WS_MAX'].mean() / hap_wind.WS_MAX.quantile(0.95)
mean_wsinsmax = hap_wind[hap_wind.WS_INS_MAX >= hap_wind.WS_INS_MAX.quantile(0.9)]['WS_INS_MAX'].mean() / hap_wind.WS_INS_MAX.quantile(0.95)
wind_freq = hap_wind[hap_wind.WS_INS_MAX >= hap_wind.WS_INS_MAX.quantile(0.9)].shape[0] / hap_wind.shape[0]

# 풍속 위험도 계산
wind_risk = 0.35*mean_wsmax + 0.35*mean_wsinsmax + 0.3*wind_freq

In [ ]:
# 나무 종류별 가중치: 침엽수(1)=0.4, 활엽수(2)=0.1, 혼효림(3)=0.15, 대나무(4)=0.35
forest_area.drop('geometry', axis=1, inplace = True)
forest_area.ADM_NM = forest_area.ADM_NM.str[:-1]
forest_area = forest_area[forest_area['fo_area'] != 0].copy()

# 단위를 km^2로 통합
forest_area['fo_area'] = forest_area['fo_area'] / 1000000

hap_forest_area = forest_area.groupby(['ADM_NM','FRTP_CD'])['fo_area'].sum().reset_index()
tree_ratio = {1 : 0.4, 2 : 0.1, 3 : 0.15 ,4 : 0.35}
hap_forest_area['tree_score'] = hap_forest_area.apply(lambda x: tree_ratio[int(x['FRTP_CD'])] * x['fo_area'],axis=1)

In [ ]:
tree_type_map = {
    '1': '침엽수림',
    '2': '활엽수림',
    '3': '혼효림',
    '4': '대나무'
}
hap_forest_area['tree_type'] = hap_forest_area['FRTP_CD'].map(tree_type_map)

# 피벗테이블 변환: 행=ADM_NM, 열=tree_type, 값=fo_area
pivot_df = hap_forest_area.pivot_table(index='ADM_NM', columns='tree_type', values='fo_area',aggfunc='sum',).fillna(0)

colors = {
    '침엽수림': '#2ca02c',
    '활엽수림': '#d62728',
    '혼효림': '#1f77b4',
    '대나무': '#ff7f0e'
}

plt.rc('font', family='Malgun Gothic')
fig, ax = plt.subplots(figsize=(12,7))
bottom = np.zeros(len(pivot_df))

for tree in ['침엽수림', '활엽수림', '혼효림', '대나무']:
    ax.bar(pivot_df.index, pivot_df[tree], bottom=bottom, label=tree, color=colors[tree])
    bottom += pivot_df[tree]

ax.set_ylabel('산림면적 (km²)')
ax.set_title('읍면별 나무종류별 산림면적 분포 (스택형 막대그래프)')
ax.legend(title='나무종류',fontsize=16)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 정규화 후 자연 요인 기반 위험도 계산
hap_nat_risk = hap_forest_area.groupby('ADM_NM')['tree_score'].sum().reset_index()
hap_nat_risk['wind_risk'] = wind_risk

scale = MinMaxScaler()
hap_nat_risk['tree_score_scaled'] = scale.fit_transform(hap_nat_risk[['tree_score']])
hap_nat_risk['nature_risk'] = hap_nat_risk['wind_risk']*0.2 + hap_nat_risk['tree_score_scaled']*0.8

### 위험도(중요지역 인접도) 계산

In [ ]:
# 국가유산청 api - 문화재 목록

url = 'http://www.khs.go.kr/cha/SearchKindOpenapiList.do'
params = {
    'ccbaCtcd': '38', # 경남
    'pageIndex': 1,
    'pageUnit': 100,
}

all_items = []
while True:
    response = requests.get(url, params=params)
    data_dict = xmltodict.parse(response.content)
    items = data_dict['result'].get('item', [])
    if not items:
        break
        
    if isinstance(items, dict):  
        items = [items]

    all_items.extend(items)
    params['pageIndex'] += 1

culture = pd.DataFrame(all_items)
# culture.to_csv('gn_culture.csv', index=False)

In [ ]:
hapcheon_culture = culture[culture['ccsiName'] == '합천군'].reset_index(drop=True)
hapcheon_culture = hapcheon_culture[hapcheon_culture['longitude'] != 0]

# 화재 피해를 입을 시 사회적, 경제적 파급력이 큰 국보, 보물인 데이터만 추출
culture_type = ['국보', '보물']
hap_imp_culture = hapcheon_culture[hapcheon_culture['ccmaName'].isin(culture_type)].copy()

hap_imp_culture['dup'] = hap_imp_culture['ccbaMnm1'].apply(lambda x : '해인사' if '해인사' in x else(
                                                           '영암사지' if '영암사지' in x else(
                                                            '청량사' if '청량사' in x else x)))

hap_uniq_culture = hap_imp_culture.drop_duplicates(subset=['dup']).reset_index(drop=True)
uniq_culture = hap_uniq_culture
uniq_culture = hap_uniq_culture[['dup','longitude','latitude']].rename(columns={'dup': 'name', 'longitude': 'lon', 'latitude': 'lat'})

In [ ]:
# haversine 라이브러리 사용하여 읍면 중심좌표, 문화재 좌표 직선거리 구하기
hap_copy = hapcheon_centroid.copy()
culture_result = []

for idx, row in hap_copy.iterrows():
    hap_xy = (row['lat'], row['lon'])
    min_dist = float('inf')
    culture_name = None

    for cul_idx, cul_row in uniq_culture.iterrows():
        cul_xy = (cul_row['lat'], cul_row['lon'])
        distance = haversine(hap_xy, cul_xy, unit='km')

        if distance < min_dist:
            min_dist = distance
            culture_name = cul_row['name']

    culture_result.append({
        'ADM_NM' : row['ADM_NM'],
        'culture_name' : culture_name,
        'min_dist' : min_dist
    })
culture_nearest = pd.DataFrame(culture_result)

scale = MinMaxScaler()
culture_nearest['min_dist_scale'] = 1 - scale.fit_transform(culture_nearest[['min_dist']]).round(6)

In [ ]:
# 합천의 인구, 면적 데이터
pop.ADM_NM = pop.ADM_NM.str[:-1]
pop['pop_density'] = pop['count'] / pop['area']
pop['log_pop_density'] = np.log(pop['pop_density'])

scale = MinMaxScaler()
pop['pop_scale'] = scale.fit_transform(pop[['log_pop_density']])

# 인구밀집도(0.6)와 읍면-문화재 최소거리(0.4)를 합하여 중요지역 인접정도에 대한 위험도 점수 계산
culture_nearest = culture_nearest.merge(pop[['ADM_NM','pop_scale']], how = 'left', on = 'ADM_NM')
culture_nearest['nearest_risk'] = culture_nearest['pop_scale']*0.6 + culture_nearest['min_dist_scale']*0.4

### 위험도(초동대응 어려운 지역) 계산

In [ ]:
# 각 읍면과 소방서와의 최소 거리
def firestation_distance(lat, lon):
    # 소방서 좌표 튜플로 변환
    fire_station_xy = tuple(zip(fire_station['경도'], fire_station['위도']))
    G = ox.graph_from_point((lat, lon), dist=10000, network_type='drive_service')  # 10km 반경, 차도만 고려

    # 좌표와 가장가까운 노드 생성
    centroid_node = ox.nearest_nodes(G, lon, lat)
    firestation_node = [ox.nearest_nodes(G, lon, lat) for lon, lat in fire_station_xy]

    dist = []

    # 노드가 없어서 거리계산이 안될경우 무한대 처리
    for node in firestation_node:
        try:
            path_length = nx.shortest_path_length(G, source=centroid_node, target=node, weight='length')
            dist.append(path_length)

        except nx.NetworkXNoPath:
            dist.append(float('inf'))

    min_distance = min(dist) / 1000  # km단위로 변환

    return min_distance


# 각 읍면별 인접 도로 개수 (10km 단위)
def count_road(lat, lon):
    G = ox.graph_from_point((lat, lon), dist=10000, network_type='drive_service')  # 10km 반경, 차도만 고려
    if G is None or len(G.edges) == 0:
        return 0

    # 도로 구간 가져오기 + 중복제거
    edge = ox.convert.graph_to_gdfs(G, nodes=False, edges=True).reset_index()
    if edge.empty or any(col not in edge.columns for col in ['v', 'key', 'u']):
        return 0

    # 소방차가 다닐 수 있는 거리만 추출
    edge['highway_car'] = edge['highway'].apply(lambda v : [v] if isinstance(v, str) else v)
    valid_highways = {'primary', 'secondary', 'tertiary'}
    filter_edge = edge[edge['highway_car'].apply(lambda list : all(x in valid_highways for x in list))]

    unique_edge = filter_edge.drop_duplicates(subset=['u','key','v'])
    edge_count = len(unique_edge)
    return edge_count

In [ ]:
hap_center['min_dist_firestation'] = hap_center.apply(lambda x : firestation_distance(x['lat'], x['lon']), axis = 1)
hap_center['road_count'] = hap_center.apply(lambda x : count_road(x['lat'], x['lon']), axis = 1)

# 용량이 너무 커서 따로 csv 파일을 저장
#hap_center.to_csv('초동대응.csv', index=False)

scale = MinMaxScaler()
hap_center['min_distance_scaled'] = scale.fit_transform(hap_center[['min_dist_firestation']])
hap_center['road_count_scaled'] = 1 - scale.fit_transform(hap_center[['road_count']])
hap_center['first_response_scaled'] = hap_center['min_distance_scaled']*0.6 + hap_center['road_count_scaled']*0.4

### 최종 위험도 데이터프레임

In [ ]:
hap_center = hap_center.merge(year_freq[['ADM_NM', 'year_score_scaled']], on='ADM_NM', how='left')
hap_center = hap_center.merge(damage_area[['ADM_NM', 'damagearea_scaled']], on='ADM_NM', how='left')

hap_center = hap_center.merge(culture_nearest[['ADM_NM','nearest_risk']], how='left', on='ADM_NM')
hap_center = hap_center.merge(hap_nat_risk[['ADM_NM','nature_risk']], how='left', on='ADM_NM')

# 빈도, 면적 부분에서 해당 읍면의 화재 데이터가 없어서 Nan 출력
hap_center.fillna(0, inplace=True)

# 최종위험도
hap_center['final_risk'] = (
    hap_center['year_score_scaled'] * 0.2 +
    hap_center['damagearea_scaled'] * 0.2 +
    hap_center['first_response_scaled'] * 0.2 +
    hap_center['nearest_risk'] * 0.2 +
    hap_center['nature_risk'] * 0.2
)

### 클러스터링

In [ ]:
# 합천 중심좌표 정규화
scale = MinMaxScaler()
hap_center[['lon_scaled', 'lat_scaled']] = scale.fit_transform(hap_center[['lon', 'lat']])

# 가중치 설정
w_risk = 0.7
w_lon = 0.15
w_lat = 0.15

# 기존 정규화된 컬럼 불러오기
lon_scaled = hap_center['lon_scaled'].to_numpy()
lat_scaled = hap_center['lat_scaled'].to_numpy()
final_risk = hap_center['final_risk'].to_numpy()

# 가중치 반영해서 새로운 X_scaled 배열 만들기
X_scaled = np.vstack([
    w_lon * lon_scaled,
    w_lat * lat_scaled,
    w_risk * final_risk
]).T

In [ ]:
# kmedoids와 kmeans 실루엣 계수 비교하기

silhouette_kmedoids = []
silhouette_kmeans = []
range_n_clusters = range(2, 11)

for k in range_n_clusters:
    kmedoids = KMedoids(n_clusters=k, random_state=42, method='pam')
    kmed_labels = kmedoids.fit_predict(X_scaled)
    kmed_score = silhouette_score(X_scaled, kmed_labels)
    silhouette_kmedoids.append(kmed_score)

for k in range_n_clusters:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans_labels = kmeans.fit_predict(X_scaled)
    kmeans_score = silhouette_score(X_scaled, kmeans_labels)
    silhouette_kmeans.append(kmeans_score)


plt.rc('font', family='Malgun Gothic')
plt.plot(range_n_clusters, silhouette_kmedoids, marker='o', color='orange', label = 'KMedoids')
plt.plot(range_n_clusters, silhouette_kmeans, marker='o', color='blue', label = 'KMeans')

# k = 4 일 때 kmedoids의 실루엣계수가 가장높음
k = 4
kmed_score_4 = silhouette_kmedoids[k - 2]
plt.text(k, kmed_score_4 + 0.01, f'{kmed_score_4:.3f}', color='orange', fontsize=12, fontweight='bold')

plt.title('Silhouette Score for KMedoids, KMeans')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.legend(loc='upper right',fontsize=15)
plt.grid(True)
plt.show()

In [ ]:
kmedoids  = KMedoids(n_clusters=4, random_state=42, method='pam')  
kmedoids.fit(X_scaled)

hap_center['cluster'] = kmedoids.labels_
cols = ['ADM_NM', 'ADM_CD', 'geometry', 'lon', 'lat', 'final_risk','cluster']
hap_center = hap_center[cols]

### PuLP를 사용하여 MCLP 진행

In [ ]:
# type(hap_center['geometry'].iloc[0])

## hap_center['geometry'] = hap_center['geometry'].apply(wkt.loads)
# gdf = gpd.GeoDataFrame(hap_center, geometry='geometry', crs='EPSG:4326')

## 최종파일 내보내기
# gdf.to_csv('hap_final_risk_kmedoids.csv',index=False)
# gdf.to_file('hap_final_risk_kmedoids.shp')

In [27]:
# df = pd.read_csv('data/hap_final_risk_kmedoids.csv')
df = hap_center.copy()

# 후보지 설정
idx = df.groupby('cluster')['final_risk'].idxmax()
facility_candidates = df.loc[idx].reset_index(drop=True)

In [ ]:
demand_points = df.copy()

def haversine_matrix(source, target):
    # 두 지점 간 거리 계산 (km)
    source_rad = np.radians(source[['lat','lon']].to_numpy())
    target_rad = np.radians(target[['lat','lon']].to_numpy())
    dist_matrix = haversine_distances(source_rad, target_rad) * 6371 
    return dist_matrix

distance_matrix = haversine_matrix(demand_points, facility_candidates)


for r in [7, 10]:
    coverage_matrix = (distance_matrix <= r).astype(int)
    for p in [3,4,5]:
        model = LpProblem('MCLP', LpMaximize)
        x_vars = [LpVariable(f'x_{j}', cat=LpBinary) for j in range(len(facility_candidates))]
        y_vars = [LpVariable(f'y_{i}', cat=LpBinary) for i in range(len(demand_points))]

        # 목적함수 : 위험도 최대화
        model += lpSum(demand_points.loc[i, 'final_risk'] * y_vars[i]for i in range(len(demand_points)))

        # 제약조건 : 반경 내 커버
        for i in range(len(demand_points)):
            model += y_vars[i] <= lpSum(coverage_matrix[i,j] * x_vars[j] for j in range(len(facility_candidates)))
        model += lpSum(x_vars) <= p

        model.solve(PULP_CBC_CMD(msg=0))

        # 결과 요약
        covered_demands = demand_points[[var.value() == 1 for var in y_vars]].copy()  # 중복 제거
        covered_risk_sum = covered_demands.final_risk.sum()
        n_selected = sum([var.value() == 1 for var in x_vars])
        a_covered = len(covered_demands)
        total_risk_sum = demand_points.final_risk.sum()
        
        coverage_ratio = covered_risk_sum / total_risk_sum if total_risk_sum > 0 else 0
        efficiency = covered_risk_sum / n_selected if n_selected > 0 else 0
        print(f"반경: {r} km, 설치개수: {p}, 위험도 커버율: {coverage_ratio:.2%}, 위험도 효율성: {efficiency:.4f}")

In [85]:
# MCLP 최적화 - 반경: 7km, 설치 개수(p): 3개
demand_points = df.copy()

def haversine_matrix(source, target):
    source_rad = np.radians(source[['lat','lon']].to_numpy())
    target_rad = np.radians(target[['lat','lon']].to_numpy())
    dist_matrix = haversine_distances(source_rad, target_rad) *  6371 # 지구 반지름
    return dist_matrix

distance_matrix = haversine_matrix(demand_points, facility_candidates)

# 커버리지 매트릭스 (거리 <= 7km)
coverage_matrix = (distance_matrix <= 7).astype(int)
p = 3

# MCLP 모델
model = LpProblem('MCLP', LpMaximize)

x_vars = [LpVariable(f'x_{j}', cat=LpBinary) for j in range(len(facility_candidates))]
y_vars = [LpVariable(f'y_{i}', cat=LpBinary) for i in range(len(demand_points))]

# 목적함수
model += lpSum(demand_points.loc[i, 'final_risk'] * y_vars[i]for i in range(len(demand_points)))

# 제약조건
for i in range(len(demand_points)):
    model += y_vars[i] <= lpSum(coverage_matrix[i,j] * x_vars[j] for j in range(len(facility_candidates)))
model += lpSum(x_vars) <= p

# 모델 최적화
model.solve(PULP_CBC_CMD(msg=0))

# 최적 선택 후보지 및 커버된 수요지점 추출
selected_facilities = facility_candidates[[var.value()==1 for var in x_vars]].copy()
covered_demands = demand_points[[var.value() == 1 for var in y_vars]].copy()

## 시각화

In [ ]:
final_hap = gpd.read_file('data/hap_final_risk_kmedoids.zip')

# (위상 오류 방지)
final_hap['geometry'] = final_hap['geometry'].buffer(0)

#좌표계 재설정 
final_hap.set_crs(epsg=5186, inplace=True, allow_override=True)  
final_hap = final_hap.to_crs(epsg=4326)  # folium, matplotlib에서 사용 가능하도록 위경도 변환

In [ ]:
# 위험도 시각화

plt.rc('font', family='Malgun Gothic') 
fig, ax = plt.subplots(figsize=(7, 10))

final_hap.plot(
    column='final_risk',          
    cmap='OrRd',                 
    linewidth=0.8,                
    edgecolor='black',          
    legend=True,                 
    legend_kwds={'label': "최종 위험도", 'shrink': 0.6},
    ax=ax
)

# 읍면 이름 라벨 표시
for idx, row in final_hap.iterrows():
    plt.annotate(
        text=row['ADM_NM'],
        xy=(row['lon'], row['lat']),
        horizontalalignment='center',
        fontsize=8,
        color='black'
    )

ax.set_title('합천군 읍면별 산불 위험도', fontsize=14)
ax.axis('off') 
plt.tight_layout()
plt.show()

In [ ]:
# 클러스터링 시각화

plt.rc('font', family='Malgun Gothic') 
fig, ax = plt.subplots(figsize=(8, 8))

# 클러스터 컬럼을 기준으로 시각화
final_hap.plot(column='cluster', categorical=True, legend=True, cmap='tab20', edgecolor='black', ax=ax)

# 읍면 이름 추가: 각 폴리곤의 중심 좌표에 이름을 표시
for idx, row in final_hap.iterrows():
    centroid = row['geometry'].centroid
    ax.text(centroid.x, centroid.y, row['ADM_NM'], fontsize=9, ha='center', color='black', fontweight='bold')

plt.title("합천군 읍면별 클러스터링 결과", fontsize=15)
plt.show()

In [ ]:
# mclp 결과 시각

plt.rc('font', family='Malgun Gothic') 

# 10km 반경 원 그리기 함수 (위경도 기준 근사)
def plot_circle(ax, lon, lat, radius_km, **kwargs):
    earth_radius = 6371  # km
    radius_deg = radius_km / earth_radius * (180 / np.pi)
    circle = plt.Circle((lon, lat), radius_deg, **kwargs)
    ax.add_patch(circle)

fig, ax = plt.subplots(figsize=(10,6))
final_hap.plot(column='final_risk', cmap='YlOrRd', legend=True,
               legend_kwds={'label': "위험도 (final_risk)", 'shrink': 0.6},
               edgecolor='black', linewidth=0.5, ax=ax, alpha=0.7)

# 설치 후보지 마커 (*)
ax.scatter(selected_gdf['lon'], selected_gdf['lat'],
           color='blue', marker='o', s=50, label='설치 후보지')

# 설치 후보지, 커버 수요지역 라벨 추가
for idx, row in selected_gdf.iterrows():
    ax.text(row['lon'] + 0.005, row['lat'] + 0.005,
            row['ADM_NM'], fontsize=10, fontweight='bold', color='blue')

# 각 설치지점에서 10km 원 표시
for idx, row in selected_gdf.iterrows():
    plot_circle(ax, row['lon'], row['lat'], 10,
                edgecolor='blue', facecolor='none', linestyle='--', linewidth=1.2, alpha=0.3)

ax.set_title("합천 MCLP 설치 후보지 및 위험도 시각화", fontsize=14)
ax.legend(loc='upper right')
ax.set_axis_off()
plt.tight_layout()
plt.show()

중심좌표는 단지 모델의 기준점일 뿐,

실제 설치에는 지형, 접근성, 토지 이용 가능성, 도로망, 수계 등 여러 요인이 고려되어야함.